<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

In [ ]:
#metodo de rayleigh


In [91]:
#metodo de iteracion en subespacio.
def gen_base_ortogonal(m,n):
    base=[]
    for j in range(m):
        b=[0.0]*n
        b[j]=1.0
        base.append(np.array(b))
    return np.array(base)
def factorizar_LU(matriz):#Codigo reciclado: Factorizacion LU
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i][k]*U[k][j]
            L[i][j]=matriz[i][j]-sum
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j][k]*U[k][i]
                U[j][i]=(matriz[j][i]-sum)/L[j][j]
            else: 
                U[i][j]=1 
                if(L[i][i]==0):
                    print("Error en la construcción de L")
    return([L,U])

def superior(list,b):#Codigo reciclado:Resolver sistema triangular
  n=len(list)
  X=[b[n-1]/list[n-1][n-1]]
  X_inv=[X[0]]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[n-i-1][n-1-j]
    X.append((b[n-1-i]-acum)/list[n-1-i][n-1-i])
    X_inv.insert(0,X[i])
  return X_inv

#Codigo reciclado:Resolver sistema triangular
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior(U_m,y))#Resolver Ux=y
def check_diag(A,tol):#Pensado para matrices en general (NO simetricas)
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)
def Jacobi_rot(A,tol,max_Iter,pr=True):
    count=0
    n=len(A)
    P=np.identity(n)
    while((check_diag(A,tol)==False)&(count<=max_Iter)):
        p,q=entrada_max(A)
        a=A[p][q]
        if(A[p][p]==A[q][q]):
            theta=np.pi/4
        else:
            theta=0.5*np.arctan(2*a/(A[p][p]-A[q][q]))
        #Ya tenemos la rotacion y podemos hacer el producto de matrices
        R=np.identity(n)
        R[p][p]=np.cos(theta)
        R[p][q]=-np.sin(theta)
        R[q][p]=-R[p][q]
        R[q][q]=R[p][p]
        A=np.dot(R.T,np.dot(A,R))#Se puede mejorar, modificando solo aquellas entradas que cambian.
        count+=1
        P=np.dot(P,R)
    if(count>max_Iter):
        print("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("num. Iteraciones: ", str(count))
    return(P,np.diagonal(A))
def entrada_max(A):#Regresa una lista [i,j], tal que a_{i,j} es el valor maximo en valor absoluto.
#Pensado para matrices simetricas
    max=abs(A[0][1])
    index=[0,1]
    for i in range(len(A)):
        for j in range(i,len(A[0])):
            if(i!=j):
                if(max<abs(A[i][j])):
                    max=abs(A[i][j])
                    index=i,j
    return(index)
def min_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    L,U=factorizar_LU(A)
    v_1=solve(L,U,v_0)#Se resuelve el sistema
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Se normaliza
        v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>=max_iter):
        return("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
        return([1/lambda_1,v_1])

In [ ]:
##Algoritmo
##1 Genera subconjunto ortonormal.
## Usa ese vector inicial (con deflación para conservar la ortogonalidad) para UNA iteración de potencia inversa (o potencia.) 
#-> eso genera una nueva matriz \Phi de approximaciones de vectores. Se normalizan. Estos nuevos vectores no necesariamente son ortogonales (solo cuando A sea ortogonal).
## Hacer el producto B=\Phi.T*A*\Phi. Parar cuando esto sea diagonal.
## Aplicar Jacobi (Completo) sobre B.
#-> Genera un producto de matrices de rotacion Q, que puedo usar para actualizar.\Phi=\Phi_old*Q.
##Repetir

In [ ]:
def quitar_contribucion(v_0,list_eigen):
    if list_eigen==[]:
        print("no se quito nada")
        return(np.array(v_0))
    else:
        aux=v_0
        for v in list_eigen:
            v_0-=np.dot(aux,np.array(v))*np.array(v)
        print("se quitaron: ",list_eigen)
        return v_0


In [ ]:
np.dot([[2,0],[0,2]],np.array([1,2]))

array([2, 4])

In [118]:
def Iter_subespacio(A,m,tol=0.000001,max_iter=4000):
    #1 Generar subconjunto ortonormal.
    phi_old=gen_base_ortogonal(m,len(A))
    B=A
    while(check_diag(B,tol=tol)==False):
        phi_new=[]
        for j in range(m):
            v_i=np.array(phi_old[j])
            if(j==0):#no hay deflacion
                v_i_next=np.linalg.solve(A,v_i)
                phi_new.append(v_i_next/np.linalg.norm(v_i_next))
            else:
                aux=v_i
                for k in range(j):#Si hay deflacion
                    v_i-=np.dot(np.array(aux),np.array(phi_new[k]))*np.array(phi_new[k])
                v_i_next=np.linalg.solve(A,v_i)
                phi_new.append(v_i_next/np.linalg.norm(v_i_next))
        B=np.dot(np.array(phi_new),np.dot(A,np.array(phi_new).T)).T
        phi_old=np.dot(np.array(phi_new).T,Jacobi_rot(A=B,tol=tol,max_Iter=max_iter,pr=False)[0]).T
    return[phi_old,np.diag(B)]

In [116]:
print("Matriz 125x125\n")
A=np.loadtxt("Eigen_125x125.txt",skiprows=1)
print("De 9 eigenpares aproximados, se calcularon sus errores y se obtuvo\n")
phi,eigenvalues=Iter_subespacio(A,m=9)
error=[]
for i in range(len(phi)):
    error.append(np.linalg.norm(np.dot(A,phi[i])-eigenvalues[i]*np.array(phi[i])))
print("maximo error:", np.max(error))
print("Matriz 125x125\n")
A=np.loadtxt("Eigen_125x125.txt",skiprows=1)
phi,eigenvalues=Iter_subespacio(A,m=25)
print("De 25 eigenpares aproximados, se calcularon sus errores y se obtuvo\n")
error=[]
for i in range(len(phi)):
    error.append(np.linalg.norm(np.dot(A,phi[i])-eigenvalues[i]*np.array(phi[i])))
print("maximo error:", np.max(error))


Matriz 125x125

De 9 eigenpares aproximados, se calcularon sus errores y se obtuvo

maximo error: 0.00011897364770181613
Matriz 125x125

De 25 eigenpares aproximados, se calcularon sus errores y se obtuvo

maximo error: 3.8335612439161984e-05


In [119]:
print("Matriz 500x500\n")
A=np.loadtxt("Eigen_500x500.txt",skiprows=1)
print("De 9 eigenpares aproximados, se calcularon sus errores y se obtuvo\n")
phi,eigenvalues=Iter_subespacio(A,m=9)
error=[]
for i in range(len(phi)):
    error.append(np.linalg.norm(np.dot(A,phi[i])-eigenvalues[i]*np.array(phi[i])))
print("maximo error:", np.max(error))
print("Matriz 500x500\n")
A=np.loadtxt("Eigen_500x500.txt",skiprows=1)
phi,eigenvalues=Iter_subespacio(A,m=25)
print("De 25 eigenpares aproximados, se calcularon sus errores y se obtuvo\n")
error=[]
for i in range(len(phi)):
    error.append(np.linalg.norm(np.dot(A,phi[i])-eigenvalues[i]*np.array(phi[i])))
print("maximo error:", np.max(error))

Matriz 500x500

De 9 eigenpares aproximados, se calcularon sus errores y se obtuvo

maximo error: 0.013601432488576351
Matriz 500x500

De 25 eigenpares aproximados, se calcularon sus errores y se obtuvo

maximo error: 0.01472776298910187


In [94]:
np.linalg.eig(A)

(array([5.97537881e-04, 5.80571977e-04, 1.86275566e-06, 8.13485613e-06,
        1.65012229e-05, 1.76249768e-05, 2.61130524e-05, 3.64801354e-05,
        3.82898652e-05, 4.84761211e-05, 4.57379733e-05, 5.61184341e-04,
        5.56494726e-04, 6.05957594e-05, 5.50804423e-04, 5.47941792e-04,
        5.43415094e-04, 5.38471098e-04, 6.80906395e-05, 7.18368801e-05,
        7.35555477e-05, 5.30048006e-04, 8.59443676e-05, 8.44370742e-05,
        9.83984375e-05, 1.00533581e-04, 1.03736601e-04, 5.18072516e-04,
        5.19857401e-04, 5.08062092e-04, 5.03446305e-04, 1.13178103e-04,
        4.99921491e-04, 4.96475222e-04, 1.26631734e-04, 1.31516893e-04,
        4.90537461e-04, 4.86156336e-04, 4.81049247e-04, 4.77670727e-04,
        1.32687792e-04, 1.35282168e-04, 1.41042173e-04, 1.42195981e-04,
        4.70812158e-04, 4.72429278e-04, 4.73347378e-04, 4.68122733e-04,
        4.65176718e-04, 4.61423601e-04, 1.55605918e-04, 4.48761475e-04,
        4.46361798e-04, 4.44778756e-04, 1.64007471e-04, 4.430839

In [12]:
#np.dot([ 0.92098861 , 0.03134517, -0.23411155],np.dot([[4,3,12],[3,10,13],[12,13,50]],[ 0.92098861 , 0.03134517, -0.23411155]))
np.linalg.norm(np.dot([[4,3,12],[3,10,13],[12,13,50]],[-0.09727447,  0.96234605, -0.25382625])-0.95*np.array([-0.09727447,  0.96234605, -0.25382625]))


5.255800913724584